In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser
from typing import List
from langchain_core.output_parsers import StrOutputParser
ak = "api-key"

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS

In [4]:
llm = OpenAI(openai_api_key="api-key")
chat_model = ChatOpenAI(openai_api_key="api-key")

In [5]:
text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

In [6]:
llm.invoke(text)

'\n\nCheerful Toes.'

In [7]:
chat_model.invoke(messages)

AIMessage(content='VibrantSox')

In [10]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

In [12]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

In [14]:
class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")

['hi', 'bye']

In [18]:
class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI(openai_api_key="api-key") | CommaSeparatedListOutputParser()
chain.invoke({"text": "colors"})

['red', 'blue', 'green', 'yellow', 'orange']

In [21]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(openai_api_key=ak)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why did the ice cream go to therapy?\nBecause it had too many toppings and couldn't find its vanilla self!"

In [22]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [23]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [24]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

In [26]:
message = model.invoke(prompt_value)

In [27]:
message

AIMessage(content='Why did the ice cream go to therapy?\n\nBecause it had too many sprinkles of anxiety!')

In [28]:
output_parser.invoke(message)

'Why did the ice cream go to therapy?\n\nBecause it had too many sprinkles of anxiety!'

In [3]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(openai_api_key=ak),
)

In [4]:
retriever = vectorstore.as_retriever()

In [5]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [6]:
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=ak)
output_parser = StrOutputParser()

In [7]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)


In [8]:
chain = setup_and_retrieval | prompt | model | output_parser

In [9]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [10]:
chain.input_schema.schema()

{'title': 'RunnableParallelInput', 'type': 'string'}

In [11]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [12]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'str

In [13]:
chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}

In [20]:
for s in chain.stream("where did harrison work?"):
    print(s, end="", flush=True)

Harrison worked at Kensho.

In [18]:
chain.batch(["where did harrison work?", "why did harrison work?"])

['Harrison worked at Kensho.',
 'Based on the given context, it is not possible to determine why Harrison worked.']

In [17]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [21]:
async for s in chain.astream("where did harrison work?"):
    print(s, end="", flush=True)

Harrison worked at Kensho.

In [22]:
await chain.ainvoke("where did harrison work?")

'Harrison worked at Kensho.'

In [24]:
await chain.abatch(["where did harrison work?"])

['Harrison worked at Kensho.']

In [28]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings(openai_api_key=ak)
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

async for chunk in retrieval_chain.astream_log(
    "where did harrison work?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '3455699e-926d-409c-a942-b8d4cb1cdfa2',
            'logs': {},
            'streamed_output': []}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': '1e79632a-b656-4b91-8f87-4a1b29a7643d',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2023-12-16T14:19:31.631',
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
            'type': 'retriever'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='harrison worked at kensho')]}},
 {'op': 'add',
  'path': '/logs/Docs/end_time',
  'value': '2023-12-16T14:19:32.025'})
------------------------

In [29]:
async for chunk in retrieval_chain.astream_log(
    "where did harrison work?", include_names=["Docs"], diff=False
):
    print("-" * 70)
    print(chunk)

----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '1b0bdb63-ddb1-45b6-bec3-058e88b81af4',
 'logs': {},
 'streamed_output': []})
----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '1b0bdb63-ddb1-45b6-bec3-058e88b81af4',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': '386d20c1-fc11-4b6d-88f2-5c30e90896ed',
                   'metadata': {},
                   'name': 'Docs',
                   'start_time': '2023-12-16T14:21:48.158',
                   'streamed_output_str': [],
                   'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
                   'type': 'retriever'}},
 'streamed_output': []})
----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '1b0bdb63-ddb1-45b6-bec3-058e88b81af4',
 'logs': {'Docs': {'end_time': '2023-12-16T14:21:48.823

In [4]:
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings(openai_api_key=ak)
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=ak)

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [9]:
from operator import itemgetter

In [6]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "italian"})

'Harrison lavorava a Kensho.'

In [7]:
model = ChatOpenAI(openai_api_key=ak)
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content="Why don't bears wear shoes? \n\nBecause they have bear feet!"),
 'poem': AIMessage(content="In the forest's embrace,\nA gentle giant roams with grace.")}

In [8]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [9]:
from langchain_core.runnables import RunnableLambda


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI(openai_api_key=ak)

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 equals 12.')

In [11]:
from langchain_core.runnables import RunnableConfig
import json

def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI(openai_api_key=ak)
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

In [13]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'foo': 'bar'}
Tokens Used: 65
	Prompt Tokens: 56
	Completion Tokens: 9
Successful Requests: 1
Total Cost (USD): $0.00010200000000000001


In [15]:
from langchain.chat_models import ChatAnthropic

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI(openai_api_key=ak)
    | StrOutputParser()
)

In [16]:
chain.invoke({"question": "how do I call Anthropic?"})

'Anthropic'

In [17]:
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(openai_api_key=ak)
)
anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(openai_api_key=ak)
)
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(openai_api_key=ak)
)

In [18]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)

In [19]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [20]:
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content="As Dario Amodei told me, using Anthropic involves a few steps. First, you need to define the problem you want to solve and determine the relevant variables in your system. Then, you can use Anthropic's modeling language to encode your system and specify the relationships between the variables.\n\nNext, you'll need to generate a distribution over the variables in your system, either by using existing data or making assumptions. You can use Anthropic's inference engine to reason about the distribution and make predictions or draw conclusions.\n\nFinally, you can use Anthropic's optimization capabilities to find the best values for the variables that satisfy certain criteria or maximize certain objectives. This can help you make decisions or optimize your system for specific outcomes.\n\nOverall, the key is to carefully define your problem, model your system, reason about the distribution of variables, and use optimization techniques to find the best solutions.")

In [21]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, to use LangChain, you need to follow these steps:\n\n1. Visit the LangChain website and create an account.\n2. Once you have logged in, you can either create a new language project or join an existing one.\n3. If you are creating a new project, you will need to provide details such as the source language, target language, project type, and any specific requirements.\n4. After setting up your project, you can start uploading your source documents or files that need to be translated.\n5. LangChain will assign professional translators who are proficient in the specific language pair and domain of your project.\n6. Translators will work on your project, and you can communicate with them through the platform if you have any questions or need clarifications.\n7. Once the translation is complete, you can review and approve the translated files.\n8. LangChain also provides additional services like proofreading and editing if you require them.\n9. F

In [22]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='The answer to 2 + 2 is 4.')

In [23]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(openai_api_key=ak,temperature=0)
runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION:
Subtracting 7 from both sides of the equation, we get:
x^3 = 12 - 7
x^3 = 5

Taking the cube root of both sides, we get:
x = ∛5

Therefore, the solution to the equation x^3 + 7 = 12 is x = ∛5.


In [25]:
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12




In [26]:
function = {
    "name": "solver",
    "description": "Formulates and solves an equation",
    "parameters": {
        "type": "object",
        "properties": {
            "equation": {
                "type": "string",
                "description": "The algebraic expression of the equation",
            },
            "solution": {
                "type": "string",
                "description": "The solution to the equation",
            },
        },
        "required": ["equation", "solution"],
    },
}

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(openai_api_key=ak, temperature=0).bind(
    function_call={"name": "solver"}, functions=[function]
)
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
runnable.invoke("x raised to the third plus seven equals 12")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "equation": "x^3 + 7 = 12",\n  "solution": "x = 1"\n}', 'name': 'solver'}})

In [29]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [31]:
model = ChatOpenAI(openai_api_key=ak,model="gpt-3.5-turbo-1106").bind(tools=tools)
model.invoke("What's the weather in SF, NYC and LA?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ovt9edsJGhsaa9qU9R8PIV3x', 'function': {'arguments': '{"location": "San Francisco, CA", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_UvLhlp1uS1MN8h92f5J7soE8', 'function': {'arguments': '{"location": "New York, NY", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_w8aVGgRLWsGIUHx4QFlOM1RO', 'function': {'arguments': '{"location": "Los Angeles, CA", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}]})

In [32]:
from langchain_core.runnables import ConfigurableField

In [33]:
model = ChatOpenAI(openai_api_key=ak, temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

In [34]:
model.invoke("pick a random number")

AIMessage(content='7')

In [35]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke("pick a random number")

AIMessage(content='42')

In [36]:
prompt = PromptTemplate.from_template("Pick a random number above {x}")
chain = prompt | model

In [37]:
chain.invoke({"x": 0})

AIMessage(content='57')

In [38]:
chain.with_config(configurable={"llm_temperature": 0.9}).invoke({"x": 0})

AIMessage(content='74')

In [39]:
from langchain.runnables.hub import HubRunnable

In [41]:
!pip install langchainhub

In [42]:
prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    owner_repo_commit=ConfigurableField(
        id="hub_commit",
        name="Hub Commit",
        description="The Hub commit to pull from",
    )
)

In [43]:
prompt.invoke({"question": "foo", "context": "bar"})

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: foo \nContext: bar \nAnswer:")])

In [44]:
prompt.with_config(configurable={"hub_commit": "rlm/rag-prompt-llama"}).invoke(
    {"question": "foo", "context": "bar"}
)

ChatPromptValue(messages=[HumanMessage(content="[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> \nQuestion: foo \nContext: bar \nAnswer: [/INST]")])

In [45]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
# Here we're going to use a bad model name to easily create a chain that will error
chat_model = ChatOpenAI(openai_api_key=ak, model_name="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()

In [46]:
prompt_template = """Instructions: You should always include a compliment in your response.

Question: Why did the {animal} cross the road?"""
prompt = PromptTemplate.from_template(prompt_template)
llm = OpenAI(openai_api_key=ak)
good_chain = prompt | llm

In [47]:
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"animal": "turtle"})

'\n\nAnswer: The turtle crossed the road to get to the other side! I admire its determination and persistence.'

In [48]:
from typing import Iterator, List

In [49]:
prompt = ChatPromptTemplate.from_template(
    "Write a comma-separated list of 5 animals similar to: {animal}"
)
model = ChatOpenAI(openai_api_key=ak, temperature=0.0)

str_chain = prompt | model | StrOutputParser()

In [50]:
for chunk in str_chain.stream({"animal": "bear"}):
    print(chunk, end="", flush=True)

lion, tiger, wolf, gorilla, panda

In [51]:
str_chain.invoke({"animal": "bear"})

'lion, tiger, wolf, gorilla, panda'

In [52]:
# This is a custom parser that splits an iterator of llm tokens
# into a list of strings separated by commas
def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
    # hold partial input until we get a comma
    buffer = ""
    for chunk in input:
        # add current chunk to buffer
        buffer += chunk
        # while there are commas in the buffer
        while "," in buffer:
            # split buffer on comma
            comma_index = buffer.index(",")
            # yield everything before the comma
            yield [buffer[:comma_index].strip()]
            # save the rest for the next iteration
            buffer = buffer[comma_index + 1 :]
    # yield the last chunk
    yield [buffer.strip()]

In [53]:
list_chain = str_chain | split_into_list

In [54]:
for chunk in list_chain.stream({"animal": "bear"}):
    print(chunk, flush=True)

['lion']
['tiger']
['wolf']
['gorilla']
['panda']


In [55]:
list_chain.invoke({"animal": "bear"})

['lion', 'tiger', 'wolf', 'gorilla', 'panda']

In [56]:
from typing import AsyncIterator

async def asplit_into_list(
    input: AsyncIterator[str]
) -> AsyncIterator[List[str]]:  # async def
    buffer = ""
    async for (
        chunk
    ) in input:  # `input` is a `async_generator` object, so use `async for`
        buffer += chunk
        while "," in buffer:
            comma_index = buffer.index(",")
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index + 1 :]
    yield [buffer.strip()]


list_chain = str_chain | asplit_into_list

In [57]:
async for chunk in list_chain.astream({"animal": "bear"}):
    print(chunk, flush=True)

['lion']
['tiger']
['wolf']
['gorilla']
['panda']


In [58]:
await list_chain.ainvoke({"animal": "bear"})

['lion', 'tiger', 'wolf', 'gorilla', 'panda']

In [60]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
model = ChatOpenAI(openai_api_key=ak)
chain = prompt | model

In [61]:
chain.invoke({"foo": "bears"})

AIMessage(content="Sure, here's a bear joke for you:\n\nWhy don't bears wear shoes?\n\nBecause they have bear feet!")

In [62]:
chain = prompt | model.bind(stop=["\n"])

In [63]:
chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears wear shoes?")

In [64]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [65]:
chain.invoke({"foo": "bears"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "setup": "Why don\'t bears wear shoes?",\n  "punchline": "Because they have bear feet!"\n}', 'name': 'joke'}})

In [66]:
chain = prompt | model | StrOutputParser()

In [67]:
chain.invoke({"foo": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

In [69]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [70]:
chain.invoke({"foo": "bears"})

{'setup': "Why don't bears use cellphones?",
 'punchline': "Because they can't bear to hear the ringtone!"}

In [71]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [72]:
chain.invoke({"foo": "bears"})

"Why don't bears like fast food?"

In [73]:
map_ = RunnableParallel(foo=RunnablePassthrough())
chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [74]:
chain.invoke("bears")

"Why don't bears wear shoes?"

In [ ]:
chain = (
    {"foo": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)
chain.invoke("bears")

In [76]:
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings(openai_api_key=ak)
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(openai_api_key=ak)

In [77]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [78]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [84]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context":  itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [85]:
chain.invoke({"question": "where did harrison work", "language": "italian"})

'Harrison ha lavorato a Kensho.'

In [89]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string

In [90]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [91]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [92]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [93]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(openai_api_key=ak, temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI(openai_api_key=ak)

In [94]:
conversational_qa_chain.invoke(
    {
        "question": "where did harrison work?",
        "chat_history": [],
    }
)

AIMessage(content='Harrison was employed at Kensho.')

In [95]:
conversational_qa_chain.invoke(
    {
        "question": "where did he work?",
        "chat_history": [
            HumanMessage(content="Who wrote this notebook?"),
            AIMessage(content="Harrison"),
        ],
    }
)

AIMessage(content='Harrison worked at Kensho.')

In [96]:
from langchain.memory import ConversationBufferMemory

In [97]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [98]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(openai_api_key=ak,temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(openai_api_key=ak),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [100]:
t = loaded_memory | standalone_question |retrieved_documents
inputs = {"question": "where did harrison work?"}
t.invoke(inputs)

{'docs': [Document(page_content='harrison worked at kensho')],
 'question': 'Where was Harrison employed?'}

{'answer': AIMessage(content='Harrison was employed at Kensho.'),
 'docs': [Document(page_content='harrison worked at kensho')]}

In [102]:
result = final_chain.invoke(inputs)

In [103]:
memory.save_context(inputs, {"answer": result["answer"].content})

In [104]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='where did harrison work?'),
  AIMessage(content='Harrison was employed at Kensho.')]}

In [105]:
inputs = {"question": "but where did he really work?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Harrison actually worked at Kensho.'),
 'docs': [Document(page_content='harrison worked at kensho')]}

In [3]:
prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

In [10]:
model = ChatOpenAI(openai_api_key=ak)

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "korean"})

'바락 오바마, 미국의 44대 대통령이 태어난 도시는 하와이의 혼룰루입니다.'

In [11]:
prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [12]:
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of There is no known fruit specifically named "Coral". and the flag of Comoros?')])

In [31]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-fWkhIhxlpHEVNhS4QCO3h34C on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [29]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI(openai_api_key=ak)
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI(openai_api_key=ak)
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI(openai_api_key=ak)
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI(openai_api_key=ak)
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [23]:
chain.invoke({"input": "scrum"})

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-fWkhIhxlpHEVNhS4QCO3h34C on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}